In [1]:
import json
import os
from dotenv import load_dotenv
load_dotenv('./.env')

from openai import OpenAI
from langchain_community.document_loaders import Docx2txtLoader
from langchain.embeddings import CacheBackedEmbeddings, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.retrievers import BM25Retriever,EnsembleRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler
from langchain.document_loaders import DirectoryLoader

from utils import * 

from datetime import datetime
import os
import re

In [12]:
# ----------------------------------------------------------------------------------------------------
# Initialize API keys
# ----------------------------------------------------------------------------------------------------
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


# ----------------------------------------------------------------------------------------------------
# Defining the LLM model
# ----------------------------------------------------------------------------------------------------
client = OpenAI()
model_name = "gpt-4-1106-preview"


# ----------------------------------------------------------------------------------------------------
# - Loading DocX file
# - Extracting Contents and Tables
# - Chunking
# ----------------------------------------------------------------------------------------------------
# from utils import extract_format_tables,DocxProcessor
# file_name = "Sample RAG_v2.docx"
file_name = input("Enter File Name: ")
file_path = f"Data\{file_name}"
tables_list = extract_format_tables(file_path) 
doc_processor = DocxProcessor(file_path,tables_list)
doc_processor.load_headings()
doc_processor.read_content()
chunks_lst = doc_processor.chunking()


# ----------------------------------------------------------------------------------------------------
# Joining all content values into one text
# ----------------------------------------------------------------------------------------------------
all_content_text = '\n'.join(item['content'] for item in chunks_lst)
print(all_content_text)


Header: None
SAMPLE 
Analisi Funzionale
CHANGE P20231458
Redatto da: __Risorsa1____		Data: _ Dicembre 2023_(Analista Funzionale)
Verificato da: _Risorsa2___		Data: _ Dicembre 2023
Approvato da: _Risorsa3____		Data: _ Dicembre 2023
Row 1: Cell 1: Storia delle modifiche | Cell 2: Storia delle modifiche | Cell 3: Storia delle modifiche
Row 2: Cell 1: Versione | Cell 2: Data | Cell 3: Descrizione modifica
Row 3: Cell 1: 1.0 | Cell 2: 13-12-2023 | Cell 3: Prima emissione

Header: Scopo e campo di applicazione
Nell’ambito del progetto “Tracciabilità del trasferimento di fondi ai fini antiriciclaggio ai sensi del Reg. (UE) 847/2015” viene richiesto di implementare nuove e aggiuntive funzioni di controllo autorizzativo volte a individuare eventuali richieste di trasferimenti di fondi che non soddisfano i requisiti di completezza dei dati informativi sull’ordinante e sul beneficiario.
Le operazioni che non conterranno tali dati informativi obbligatori, o che non supereranno i controlli formali 

In [3]:
# ----------------------------------------------------------------------------------------------------
# Converting the list of JSON chunks into a list of Document objects
# ----------------------------------------------------------------------------------------------------
json_list = chunks_lst  # Replace this with your actual list of JSON objects
document_list = []
for json_data in json_list:
    document = json_to_document(json_data)
    document_list.append(document)
document_list

[Document(page_content='Header: None\nSAMPLE \nAnalisi Funzionale\nCHANGE P20231458\nRedatto da: __Risorsa1____\t\tData: _ Dicembre 2023_(Analista Funzionale)\nVerificato da: _Risorsa2___\t\tData: _ Dicembre 2023\nApprovato da: _Risorsa3____\t\tData: _ Dicembre 2023\nRow 1: Cell 1: Storia delle modifiche | Cell 2: Storia delle modifiche | Cell 3: Storia delle modifiche\nRow 2: Cell 1: Versione | Cell 2: Data | Cell 3: Descrizione modifica\nRow 3: Cell 1: 1.0 | Cell 2: 13-12-2023 | Cell 3: Prima emissione\n', metadata={'source': 'Data\\Sample RAG_v2.docx', 'heading': None, 'chunk_num': 1}),
 Document(page_content='Header: Scopo e campo di applicazione\nNell’ambito del progetto “Tracciabilità del trasferimento di fondi ai fini antiriciclaggio ai sensi del Reg. (UE) 847/2015” viene richiesto di implementare nuove e aggiuntive funzioni di controllo autorizzativo volte a individuare eventuali richieste di trasferimenti di fondi che non soddisfano i requisiti di completezza dei dati informat

### **Note:** Now we have a **LIST OF CHUNKS** (document_list) and a **FULL CONTENT TEXT** (all_content_text)

Next steps are:
- Embedding & Sinking of **LIST OF CHUNKS** into a VectorDB
- Summerazing the **FULL CONTENT TEXT** ===>> **LIST OF MACRO TESTS**
- Extraction and Sorting of **CHUNKS from VectorDB** for each Macro Test
- Generation of **Micro Tests** for each Macro Test

In [6]:
# ----------------------------------------------------------------------------------------------------
# Embedding & Sinking of **LIST OF CHUNKS** into a VectorDB
# ----------------------------------------------------------------------------------------------------
embed_model_id = 'text-embedding-ada-002'
core_embeddings_model = OpenAIEmbeddings(model=embed_model_id)
vectorstore = FAISS.from_documents(document_list,core_embeddings_model)

In [10]:
file_path[5:-5]

'Sample RAG_v2'

In [15]:
version = input(f"Enter Index Version Number for the file {file_path}: ") #version for index in \Store
save_directory = f"Store/{file_path[5:-5]}_v{version}"
save_directory
vectorstore.save_local(save_directory)

In [18]:
# ----------------------------------------------------------------------------------------------------
# Prompt for full text summarization
# ----------------------------------------------------------------------------------------------------
prompt = f'''
        Sarà fornita una documentazione di Analisi Funzionale che include le modifiche o gli aggiornamenti implementati per un software o un servizio. 

        Compito:
        Il tuo obiettivo è individuare tutti aggiornamenti implementati e creare il più possibile casi di macro test.
        Riassumi tutti i casi di test possibili nei seguenti campi:
        - 'Titolo: titolo per il Test Case',
        - 'Descrizione: breve descrizione del cambiamento implementato',
        - 'Obiettivo: obiettivo del test',
        - 'Output atteso: fornire un output atteso se è menzionato nella documentazione',
        - 'Input atteso: fornire un input atteso se è menzionato nella documentazione',
        - 'Keywords: estrai keywords di Test Case'
        Ricorda inoltre di mantenere tutte le date e i valori numeriche nel tuo riassunto.
        Ritorna solo la lista dei Casi di Test.

        Analisi Funzionale:
                {all_content_text}.

        Riassunti:
'''

print(prompt)


        Sarà fornita una documentazione di Analisi Funzionale che include le modifiche o gli aggiornamenti implementati per un software o un servizio. 

        Compito:
        Il tuo obiettivo è individuare tutti aggiornamenti implementati e creare il più possibile casi di micro test.
        Riassumi tutti i casi di test possibili nei seguenti campi:
        - 'Titolo: titolo per il Test Case',
        - 'Descrizione: breve descrizione del cambiamento implementato',
        - 'Obiettivo: obiettivo del test',
        - 'Output atteso: fornire un output atteso se è menzionato nella documentazione',
        - 'Input atteso: fornire un input atteso se è menzionato nella documentazione',
        - 'Keywords: estrai keywords di Test Case'
        Ricorda inoltre di mantenere tutte le date e i valori numeriche nel tuo riassunto.
        Ritorna solo la lista dei Casi di Test.

        Analisi Funzionale:
                Header: None
SAMPLE 
Analisi Funzionale
CHANGE P20231458
Redatto da: 

In [19]:
# ----------------------------------------------------------------------------------------------------
# Number of tokens for FULL CONTENT TEXT
# ----------------------------------------------------------------------------------------------------
print(num_tokens_from_string(prompt, "cl100k_base"))

4189


In [20]:
# ----------------------------------------------------------------------------------------------------
# Asking ChatGPT to summerize the FULL CONTENT TEXT into MAcro tests
# ----------------------------------------------------------------------------------------------------
response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": f"Content: {prompt}"}],
    temperature=0.0
)

In [22]:
print(response.choices[0].message.content)

1. Titolo: Verifica Reg487 Verification Result
   Descrizione: Controllo dell'esito dei controlli formali effettuati da Autorizzativo.
   Obiettivo: Assicurarsi che il campo "Reg487 Verification Result" sia valorizzato correttamente.
   Output atteso: 0 = Tutti i controlli Reg487 superati, 1-6 = Specifico errore di validazione.
   Input atteso: Valori da 0 a 6.
   Keywords: Reg487 Verification Result, controllo formale, Autorizzativo, validazione.

2. Titolo: Controllo Trasferimento Fondi infra UE
   Descrizione: Verifica della presenza del campo "Cod. unico di identificazione operazione" per trasferimenti infra UE.
   Obiettivo: Verificare l'obbligatorietà del campo per trasferimenti infra UE.
   Output atteso: Campo obbligatorio valorizzato.
   Input atteso: Cod. unico di identificazione operazione.
   Keywords: Trasferimento fondi, infra UE, codice identificativo, obbligatorio.

3. Titolo: Controllo Trasferimento Fondi extra UE < 1000,00 euro
   Descrizione: Verifica della presenza 

In [24]:
# ----------------------------------------------------------------------------------------------------
# Saving the response from ChatGPT
# ----------------------------------------------------------------------------------------------------
# Set the folder path
full_text_results_folder_path = f'./full_text_results/{model_name}'

# Check if the folder exists, create it if not
if not os.path.exists(full_text_results_folder_path):
    os.makedirs(full_text_results_folder_path)

# Get a list of all files in the folder
files = os.listdir(full_text_results_folder_path)

# Filter files that match the pattern "model_name_example_vX.txt"
pattern = re.compile(f'{file_path[5:-5]}_example_v(\d+)\.txt')
matching_files = [file for file in files if pattern.match(file)]

# If there are matching files, find the latest version
if matching_files:
    latest_version = max(int(pattern.match(file).group(1)) for file in matching_files)
else:
    latest_version = 0  # If no matching files found, start from version 0

# Increment the version for the new file
new_version = latest_version + 1

# Form the new file name
full_text_result_file_name = f'{file_path[5:-5]}_example_v{new_version}.txt'

# Open a file in write mode ('w') with the new file name
with open(os.path.join(full_text_results_folder_path, full_text_result_file_name), 'w') as file:
    # Write content to the file
    file.write(response.choices[0].message.content)

In [26]:
# ----------------------------------------------------------------------------------------------------
# Reading and splitting the response from ChatGPT into macro_tests
# ----------------------------------------------------------------------------------------------------
file_path = os.path.join(full_text_results_folder_path, full_text_result_file_name)  # Replace 'your_file.txt' with the actual file path
# file_path = "C:\\Users\\ElshanChalabiyev\\Desktop\\NEXI to read\\AI solutions\\Development\\RAG\\full_text_results\\gpt-4-1106-preview\\gpt-4-1106-preview_example_v11.txt"
with open(file_path, 'r') as file:
    # Read the entire contents of the file
    sum_res = file.read()

# Splitting the text into paragraphs
macro_tests = re.split(r'\n\s*\n', sum_res)
macro_tests


['1. Titolo: Verifica Reg487 Verification Result\n   Descrizione: Controllo dell\'esito dei controlli formali effettuati da Autorizzativo.\n   Obiettivo: Assicurarsi che il campo "Reg487 Verification Result" sia valorizzato correttamente.\n   Output atteso: 0 = Tutti i controlli Reg487 superati, 1-6 = Specifico errore di validazione.\n   Input atteso: Valori da 0 a 6.\n   Keywords: Reg487 Verification Result, controllo formale, Autorizzativo, validazione.',
 '2. Titolo: Controllo Trasferimento Fondi infra UE\n   Descrizione: Verifica della presenza del campo "Cod. unico di identificazione operazione" per trasferimenti infra UE.\n   Obiettivo: Verificare l\'obbligatorietà del campo per trasferimenti infra UE.\n   Output atteso: Campo obbligatorio valorizzato.\n   Input atteso: Cod. unico di identificazione operazione.\n   Keywords: Trasferimento fondi, infra UE, codice identificativo, obbligatorio.',
 "3. Titolo: Controllo Trasferimento Fondi extra UE < 1000,00 euro\n   Descrizione: Ver

In [29]:
# core_embeddings_model, macro_tests

# ----------------------------------------------------------------------------------------------------
# Create a folder with the current date as its name
# ----------------------------------------------------------------------------------------------------
# Get the current date
current_date = datetime.now().strftime("%Y-%m-%d")
# Create a folder with the current date as its name
micro_tests_results_folder_path = os.path.join(".\\micro_tests_results", current_date)
try:
    os.makedirs(micro_tests_results_folder_path)
    print(f"Folder '{current_date}' created successfully.")
except OSError as e:
    print(f"Error creating folder: {e}")


for i,test in enumerate(macro_tests):
    # ----------------------------------------------------------------------------------------------------
    # Embedding query for Similarity search
    # ----------------------------------------------------------------------------------------------------
    query = f"Trova tutto che sai riguardo a questo test {test}"
    embedding_vector = core_embeddings_model.embed_query(query)

    # ----------------------------------------------------------------------------------------------------
    # Extraction and Sorting of **CHUNKS from VectorDB** for each Macro Test
    # ----------------------------------------------------------------------------------------------------
    docs_resp = vectorstore.similarity_search_by_vector(embedding_vector,k=7)
    # Sorting the list of documents based on the 'chunk_number' attribute
    sorted_documents = sorted(docs_resp, key=lambda x: x.metadata['chunk_num'])

    # ----------------------------------------------------------------------------------------------------
    # Format the prompt for Micro tests generation
    # ----------------------------------------------------------------------------------------------------
    retrieved_chunks = [doc.page_content for doc in sorted_documents]
    chunks_formatted = "\n\n".join(retrieved_chunks)
    # print(chunks_formatted)

    micro_test_template = f'''
    Act as a Functional Analyst who creates Test Cases for Test team. 
    You are provided with a summarized text of Macro test and a context from the Functional analysis documentation where a new implementation or update to a software or service is described.
    Your goal is to create detailed micro tests from the provided context related only to the provided Macro Test, preserving important numbers and values. 

    Do provide only helpful answers, using only information from the provided contexts.
    Format the new test cases with the following fields:
        - 'Titolo: titolo di Micro Test Case',
        - 'Descrizione: descrizione del cambiamento implementato',
        - 'Obiettivo: obiettivo dettagliato del test',
        - 'Input atteso: fornire un input atteso se è menzionato nella documentazione',
        - 'Output atteso: fornire un output atteso se è menzionato nella documentazione',
        - 'Procedura: procedura di svolgemento di test step by step'
    Please do not forget that you should create as much as possible Micro Tests!
    Summarized Macro Test: 
    {test}
    Context: 
    {chunks_formatted}

    Micro test cases:
    '''
    print("Query token size:",num_tokens_from_string(micro_test_template, "cl100k_base"))
    # print(micro_test_template)

    # ----------------------------------------------------------------------------------------------------
    # Micro tests generation
    # ----------------------------------------------------------------------------------------------------
    generated_micro_tests = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": f"Content: {micro_test_template}"}],
        temperature=0.0
    )

    # ----------------------------------------------------------------------------------------------------
    # Open a file in write mode ('w')
    # ----------------------------------------------------------------------------------------------------
    with open(os.path.join(micro_tests_results_folder_path,f'{i}_micro_tests.txt'), 'w') as file:
        # Write content to the file
        file.write("#"*50+"\nMACRO TEST\n"+ test+"\n"+"#"*50+ "\n\n"+generated_micro_tests.choices[0].message.content)

Folder '2024-01-26' created successfully.
Query token size: 4092
Query token size: 4008
Query token size: 4129
Query token size: 4162
Query token size: 4106
Query token size: 4142


In [47]:
print(generated_micro_tests.choices[0].message.content)

After reviewing the provided micro test cases for various macro test cases, it appears that there are no micro tests with identical Titles, Descriptions, or Goals under the same conditions. Each micro test case is unique to its corresponding macro test case and is designed to test specific conditions and expected outcomes related to the macro test case's objectives.

Therefore, there are no references to provide for micro tests with identical Titles, Descriptions, or Goals, as each micro test is distinct and tailored to the requirements of the macro test it is associated with.


In [41]:
for filename in os.listdir(micro_tests_results_folder_path):
    print(filename)

0_micro_tests.txt
1_micro_tests.txt
2_micro_tests.txt
3_micro_tests.txt
4_micro_tests.txt
5_micro_tests.txt


In [44]:
import os
import shutil


# path = r".\micro_tests_results\2024-01-26"
# output_file_version = input("Enter the version nymber for output file: ")
output_file = f"{micro_tests_results_folder_path}\combined_text.txt"

with open(output_file, 'w') as outfile:
    for filename in os.listdir(micro_tests_results_folder_path):
        if filename.endswith(".txt") and filename != "combined_text.txt":
            file_path = os.path.join(micro_tests_results_folder_path, filename)
            with open(file_path, 'r') as infile:
                shutil.copyfileobj(infile, outfile)
                outfile.write('\n')  # Add a newline between each file's content


In [39]:
with open(output_file, 'r') as file:
    # Read the entire contents of the file
    txt = file.read()

print("Query token size:",num_tokens_from_string(txt, "cl100k_base"))
# print(micro_test_template)
print(txt)

Query token size: 13338
##################################################
MACRO TEST
1. Titolo: Verifica Reg487 Verification Result
   Descrizione: Controllo dell'esito dei controlli formali effettuati da Autorizzativo.
   Obiettivo: Assicurarsi che il campo "Reg487 Verification Result" sia valorizzato correttamente.
   Output atteso: 0 = Tutti i controlli Reg487 superati, 1-6 = Specifico errore di validazione.
   Input atteso: Valori da 0 a 6.
   Keywords: Reg487 Verification Result, controllo formale, Autorizzativo, validazione.
##################################################

Based on the provided Macro Test and context, here are the detailed micro test cases:

1. Titolo: Verifica Reg487 Verification Result - Codice 0
   Descrizione: Verifica che il campo "Reg487 Verification Result" sia valorizzato con 0 quando tutti i controlli Reg487 sono superati.
   Obiettivo: Assicurarsi che il valore 0 sia assegnato correttamente quando non ci sono errori di validazione.
   Input atteso: 

In [45]:
prompt = f'''
    Below you are provided by lists of Micro tests for various Macro test cases. 
    Your goal is to pass over each micro tests and find micro tests that have identical Titles, Descriptions or Goals with the same conditions.
    For each similar microtest give the reference to the macro tests in which they are presented
    Content:
    {txt}
'''
# ----------------------------------------------------------------------------------------------------
# Micro tests generation
# ----------------------------------------------------------------------------------------------------
duplication_check_response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": f"Content: {prompt}"}],
    temperature=0.0
)

In [46]:
print(duplication_check_response.choices[0].message.content)

After reviewing the provided micro test cases for various macro test cases, it appears that there are no micro tests with identical Titles, Descriptions, or Goals under the same conditions. Each micro test case is unique to its corresponding macro test case and is designed to test specific conditions and expected outcomes related to the macro test case's objectives.

Therefore, there are no references to provide for micro tests with identical Titles, Descriptions, or Goals, as each micro test is distinct and tailored to the requirements of the macro test it is associated with.


In [4]:
file_path = "C:\\Users\\ElshanChalabiyev\\Desktop\\NEXI to read\\AI solutions\\Development\\RAG\\app.py"
# file_path = "C:\\Users\\ElshanChalabiyev\\Desktop\\NEXI to read\\AI solutions\\Development\\RAG\\full_text_results\\gpt-4-1106-preview\\gpt-4-1106-preview_example_v11.txt"
with open(file_path, 'r') as file:
    # Read the entire contents of the file
    script = file.read()

print(script)

#!/usr/bin/env python
# coding: utf-8

# In[79]:


import json
import os
from dotenv import load_dotenv
load_dotenv('./.env')

from openai import OpenAI
from langchain_community.document_loaders import Docx2txtLoader
from langchain.embeddings import CacheBackedEmbeddings, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.retrievers import BM25Retriever,EnsembleRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler
from langchain.document_loaders import DirectoryLoader

from utils import * 


import os
import re


# In[80]:


# ----------------------------------------------------------------------------------------------------
# Initialize API keys
# ----------------------------------------------------------------------------------------------------
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
os.environ[

In [5]:
prompt = f'''
    In the code provided below i am using:
    1. GPT4 technology to create macro tests passing full functional anlysis documentation.
    2. I am also chunking (each chunk is a heading text coming after it untill the next heading) this document and putting it into a FAISS vector database.
    3. After for each macrotest generated in step 1 i am performing similarity search in my vector db to extract chunks (generated in step 2) related to this test.
    4. concating the macro test with the content extracted from the db into a one text and passing it to GPT4 to create Micro tests for the given Macrotest and knowledge base.

    I should present this project to my boss and team and i need you to prepare presentation for it explaining each technology used in it.
    You should notice that my boss and team have no ideas regarding these technologies, so be comprehensive.
    take into attention the following topics to mention in you presentation:
    1.GPT4
    2.Token
    3.Token Limitation
    4.GPT4 Pricing
    5.Embeddings
    6.Vector Database
    Add topic that are missed and could be useful to mention
    Order topics as you think it would be better.   
    Code:
    {script}
'''
# ----------------------------------------------------------------------------------------------------
# Micro tests generation
# ----------------------------------------------------------------------------------------------------
presentation_response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": f"Content: {prompt}"}],
    temperature=0.0
)

In [7]:
print(presentation_response.choices[0].message.content)

# Presentation: Implementing GPT-4 for Macro and Micro Test Generation

## Introduction
Good [morning/afternoon/evening], everyone. Today, I'm excited to present a project that leverages cutting-edge AI technology to revolutionize the way we create and manage software tests. Our project uses GPT-4, a state-of-the-art language model, to generate macro tests from functional analysis documentation. We then chunk this documentation and store it in a FAISS vector database for efficient retrieval. This allows us to extract relevant chunks for each macro test and use them to generate detailed micro tests. Let's dive into the technologies that make this possible.

## 1. GPT-4
- **What is GPT-4?**
  - GPT-4, or Generative Pre-trained Transformer 4, is an advanced AI language model developed by OpenAI.
  - It's capable of understanding and generating human-like text, making it a powerful tool for a variety of natural language processing tasks.

- **How are we using GPT-4?**
  - We use GPT-4 to a

In [8]:
with open("presentation.txt", 'w') as file:
    # Write content to the file
    file.write(presentation_response.choices[0].message.content)

In [ ]:
from docx import Document
file_name = "Data\\Sample RAG_v2.docx"
doc = Document(file_name)

In [36]:
!pip install docx2txt


[notice] A new release of pip available: 22.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
import docx2txt

# file_path = "path/to/your/document.docx"  # Replace with the actual path to your Word document
my_text = docx2txt.process(file_name)
print(my_text)


SAMPLE 

Analisi Funzionale

CHANGE P20231458



















































Redatto da: __Risorsa1____		Data: _ Dicembre 2023_(Analista Funzionale)



Verificato da: _Risorsa2___		Data: _ Dicembre 2023



Approvato da: _Risorsa3____		Data: _ Dicembre 2023





Storia delle modifiche

Versione

Data

Descrizione modifica

1.0

13-12-2023

Prima emissione










Sommario

		1	Scopo e campo di applicazione	3

		2	Definizioni e acronimi	3

		3	Riferimenti	3

		4	Architettura del Sistema	3

		5	Descrizione Funzionale	3

		5.1	Contesto di riferimento	3

		5.2	Vincoli e assunzioni	4

		6	Requisiti funzionali	4

		6.1	Gestione dati aggiuntivi su messaggio ISO	4

		6.2	Mapping Campi ISO con Tabella TR	6

		6.3	Integrazione dati Trasferimento Fondi nella GUI Sistema Antifrode	7

		6.4	Risposte Sistema Antifrode	10












Scopo e campo di applicazione

Nell’ambito del progetto “Tracciabilità del trasferimento di fondi ai fini antiriciclaggio ai sensi del Reg. (UE) 847